# 自动求导

## PyTorch中backward()方法详解
**1. backward()方法介绍**
- 在PyTorch中,对于一个计算图(Computational Graph),我们可以调用`.backward()`方法来进行反向传播,计算每个变量的梯度。
- `.backward()`方法需要从计算图中指定的某个变量开始,按照链式法则自动计算并累积每个张量相对于其输入的梯度。计算图中所有叶子节点的`.grad`属性就存储了最终的梯度。

**2. 使用backward()计算梯度**

要使用`.backward()`计算梯度,主要分以下几步:

- 构建包含可导变量(requires_grad=True)的计算图
- 根据计算图进行前向传播,计算输出
- 调用输出的`.backward()`方法启动反向传播
- 计算图中可导变量的`.grad`属性包含了梯度

例如:

```python
    x = torch.tensor(..., requires_grad=True)
    y = 2 * x + 3
    z = y**2
    z.backward()
    print(x.grad) # 4*x*2
```



**3. backward()方法的重要参数**
- `retain_graph=True`:保留计算图进行多次反向传播
- `create_graph=True`:进行高阶导数计算

利用这些参数可以实现一些复杂的梯度计算技巧。

**4. 总结**
- .backward()自动计算梯度,是PyTorch的核心功能
- 正确使用可以减少大量手动求导劳动
- 需要理解计算图、链式法则等原理,才能灵活应用

In [4]:
import torch
x = torch.arange(4.0)
print(x)

tensor([0., 1., 2., 3.])


In [7]:
# 反向传播中计算梯度,默认False
x.requires_grad=True
print(x.grad)#默认为None

None


In [11]:
y = 2 * torch.dot(x, x)
print(y)

tensor(28., grad_fn=<MulBackward0>)


In [12]:
y.backward()
print(x.grad)

tensor([ 0.,  4.,  8., 12.])


In [14]:
x.grad == 4 * x

tensor([True, True, True, True])

In [15]:
# 梯度清零
x.grad.zero_()
y = x.sum()
y.backward()
print(x.grad)

tensor([1., 1., 1., 1.])


In [20]:
x.grad.zero_()
y = x * x
y.sum().backward()
print(x.grad)

tensor([0., 2., 4., 6.])


In [22]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

## PyTorch中detach()的作用
detach()是PyTorch中非常重要的一个操作,它可以从计算图中分离出一个Tensor,使其不参与梯度反向传播。具体总结如下:

- detach()会返回一个新的Tensor,它与原Tensor共享数据,但已经从计算图中分离
- 新Tensor不再依赖计算图,所以在反向传播中,到它这就不会再递归计算梯度了
- 因此可以通过detach()防止某些Tensor的梯度计算和更新
- 如果只想断开某个中间变量的依赖,可以对其调用.detach()
- detach()返回的Tensor还在同一个设备上,没有复制数据
- 注意detach后就无法再计算这个Tensor的梯度了,因为已经从计算图分离
- detach()不同于requires_grad=False,后者可以再打开求梯度,但detach()是永久分离
- 正确使用detach可以提高效率,避免不必要的梯度计算

所以detach()非常适合在以下场景中使用:
- 冻结模型参数,防止更新
- 断开不需要进行梯度回传的中间变量
- 在纯前向推断时增加效率
  
***需要注意的是,只要需要继续求梯度,就不能detach,否则会导致梯度无法回传。***

总之,detach()是计算图和自动求导中的一个非常重要的操作,合理利用可以让训练更高效。

In [23]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [24]:
def f(a):
    b = a * 2
    # b的向量模长小于1000时
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d/a

tensor(True)